In [1]:
import pandas as pd
import pickle

In [2]:
file_path = "/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codeT5_embeddings/df_dc_embeded_by_line_base.pkl"
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

/var/folders/d8/cr798cb57kzd5kz_r8g8jm380000gn/T/ipykernel_30036/2582950095.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(fh)


In [3]:
df.head()

,sample_id,severity,method,label,embeded_sequence_sum,embeded_sequence_avg
0,8077,critical,public class DruidPooledCallableStatement exte...,1,0 -73.317101 1 -42.687691 2 -150...,0 -0.078666 1 -0.045802 2 -0.16141...
1,5553,critical,public interface IAopReferenceModel {\n\t\n\tv...,1,0 -0.477835 1 0.831359 2 -7.71129...,0 -0.020775 1 0.036146 2 -0.33527...
2,9341,critical,@XmlAccessorType(XmlAccessType.FIELD)\n@XmlTyp...,1,0 -4.968647 1 -1.361101 2 -4.72...,0 -0.134288 1 -0.036787 2 -0.12759...
3,10419,critical,public class Ipam {\n private String driver;\...,1,0 -2.197651 1 -6.160445 2 -5.59...,0 -0.032318 1 -0.090595 2 -0.08222...
4,12232,critical,public class DruidPooledCallableStatement exte...,1,0 -72.336136 1 -41.620876 2 -147...,0 -0.077614 1 -0.044658 2 -0.15841...


In [4]:
embeded_sum = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_sum']]
embeded_avg = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_avg']]
embeded_sum.head()

,sample_id,severity,label,embeded_sequence_sum
0,8077,critical,1,0 -73.317101 1 -42.687691 2 -150...
1,5553,critical,1,0 -0.477835 1 0.831359 2 -7.71129...
2,9341,critical,1,0 -4.968647 1 -1.361101 2 -4.72...
3,10419,critical,1,0 -2.197651 1 -6.160445 2 -5.59...
4,12232,critical,1,0 -72.336136 1 -41.620876 2 -147...


In [5]:
print(embeded_sum["embeded_sequence_sum"].head())
print(embeded_sum["embeded_sequence_sum"].apply(type).value_counts())

0    0      -73.317101
1      -42.687691
2     -150...
1    0     -0.477835
1      0.831359
2     -7.71129...
2    0      -4.968647
1      -1.361101
2      -4.72...
3    0      -2.197651
1      -6.160445
2      -5.59...
4    0      -72.336136
1      -41.620876
2     -147...
Name: embeded_sequence_sum, dtype: object
embeded_sequence_sum
<class 'pandas.core.series.Series'>    2250
<class 'NoneType'>                     1444
Name: count, dtype: int64


In [6]:
def process_embedding(x):
    if x is None:
        return []  
    elif isinstance(x, pd.Series):
        return " ".join(map(str, x))
    else:
        return str(x)  

embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(process_embedding)


In [7]:
print(embeded_sum["embeded_sequence_sum"].head())

0    -73.31710052490234 -42.68769073486328 -150.438...
1    -0.47783464193344116 0.8313590884208679 -7.711...
2    -4.968647003173828 -1.3611005544662476 -4.7210...
3    -2.197650909423828 -6.160444736480713 -5.59121...
4    -72.33613586425781 -41.62087631225586 -147.642...
Name: embeded_sequence_sum, dtype: object


In [8]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [9]:
import re

def parse_flat_embedding(embedding_str):
    # Ensure input is a string
    if not isinstance(embedding_str, str):
        return []
    try:
        # Extract numerical values from the string
        values = re.findall(r"[-+]?\d*\.\d+|\d+", embedding_str)
        return [float(v) for v in values]
    except Exception as e:
        print(f"Error parsing: {embedding_str} -> {e}")
        return []


In [10]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(parse_flat_embedding)

In [11]:
n = len(embeded_sum["embeded_sequence_sum"][0])
print(f"Embedding length: {n}")

embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")


Embedding length: 768
Most common sequence length: 768
Count of rows with this length: 2227


In [12]:
 # Function to handle the trimming or padding of the sequence
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_sum["embeded_sequence_sum"].head())

0    [-73.31710052490234, -42.68769073486328, -150....
1    [-0.47783464193344116, 0.8313590884208679, -7....
2    [-4.968647003173828, -1.3611005544662476, -4.7...
3    [-2.197650909423828, -6.160444736480713, -5.59...
4    [-72.33613586425781, -41.62087631225586, -147....
Name: embeded_sequence_sum, dtype: object


In [13]:
embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
print(embed_lengths.value_counts())


embeded_sequence_sum
768    3694
Name: count, dtype: int64


In [14]:
# Create column names for embeddings
columns = [f'emb_{i+1}' for i in range(n)]

# Create a DataFrame for embedding values
data = pd.DataFrame(embeded_sum["embeded_sequence_sum"].to_list(), columns=columns)

# Add metadata columns
data['label'] = embeded_sum['label']
data['sample_id'] = embeded_sum['sample_id']
data['severity'] = embeded_sum['severity']

# Display the result
print(data.head())


       emb_1      emb_2       emb_3       emb_4      emb_5       emb_6  \
0 -73.317101 -42.687691 -150.438019  103.039841  35.124355  140.111664   
1  -0.477835   0.831359   -7.711295    4.890903  -0.823007    4.241571   
2  -4.968647  -1.361101   -4.721052    5.020623   1.029872    2.879137   
3  -2.197651  -6.160445   -5.591215    9.961710  -1.178440    1.964401   
4 -72.336136 -41.620876 -147.642883  101.578682  38.796505  128.714645   

      emb_7      emb_8      emb_9     emb_10  ...    emb_762     emb_763  \
0 -2.328502  76.979721  -7.235710 -77.745911  ... -19.821392  239.506317   
1  4.829412   3.451910   0.837943  -2.760350  ...   0.750222    5.456442   
2  0.307737   3.433656   0.364186  -4.503881  ...   0.244199    5.047576   
3 -4.029890   2.628404   3.564538  -0.590118  ...  -4.493562   14.434325   
4 -3.810141  86.228577 -15.177495 -73.152466  ... -16.878183  235.330811   

      emb_764     emb_765    emb_766     emb_767     emb_768  label  \
0 -121.746979   99.123787  

In [15]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codeT5_embeddings/processed_embeddings/codeT5_base_embed_line_by_line_sum.pkl'
pd.to_pickle(data, dataset_path)

In [16]:
print(embeded_avg["embeded_sequence_avg"].head())
print(embeded_avg["embeded_sequence_avg"].apply(type).value_counts())

0    0     -0.078666
1     -0.045802
2     -0.16141...
1    0     -0.020775
1      0.036146
2     -0.33527...
2    0     -0.134288
1     -0.036787
2     -0.12759...
3    0     -0.032318
1     -0.090595
2     -0.08222...
4    0     -0.077614
1     -0.044658
2     -0.15841...
Name: embeded_sequence_avg, dtype: object
embeded_sequence_avg
<class 'pandas.core.series.Series'>    2250
<class 'NoneType'>                     1444
Name: count, dtype: int64


In [17]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [18]:
n = len(embeded_avg["embeded_sequence_avg"][0])
print(f"Embedding length: {n}")

Embedding length: 15694


In [19]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(parse_flat_embedding)

In [20]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")

Most common sequence length: 768
Count of rows with this length: 1481


In [21]:
# Function to handle the trimming or padding of the sequence
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_avg["embeded_sequence_avg"].head())

0    [-0.07866641879081726, -0.045802243053913116, ...
1    [-0.02077541872859001, 0.036146048456430435, -...
2    [-0.1342877596616745, -0.03678650036454201, -0...
3    [-0.032318394631147385, -0.09059477597475052, ...
4    [-0.07761388272047043, -0.044657591730356216, ...
Name: embeded_sequence_avg, dtype: object


In [22]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
print(embed_lengths.value_counts())

embeded_sequence_avg
768    3694
Name: count, dtype: int64


In [23]:
columns = [f'emb_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_avg["embeded_sequence_avg"].to_list(), columns=columns)
data['label'] = embeded_avg['label']
data['sample_id'] = embeded_avg['sample_id']
data['severity'] = embeded_avg['severity']
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-0.078666,-0.045802,-0.161414,0.110558,0.037687,0.150334,-0.002498,0.082596,-0.007764,-0.083418,...,-0.074862,-0.004713,-0.021268,0.256981,-0.130630,0.106356,0.061046,1,8077,critical
1,-0.020775,0.036146,-0.335274,0.212648,-0.035783,0.184416,0.209974,0.150083,0.036432,-0.120015,...,0.032618,0.237237,-0.105299,0.229636,0.026133,0.267945,-0.175635,1,5553,critical
2,-0.134288,-0.036787,-0.127596,0.135693,0.027834,0.077815,0.008317,0.092802,0.009843,-0.121727,...,-0.028849,0.006600,0.136421,-0.064934,0.206761,-0.001114,0.353806,1,9341,critical
3,-0.032318,-0.090595,-0.082224,0.146496,-0.017330,0.028888,-0.059263,0.038653,0.052420,-0.008678,...,-0.066082,0.212269,-0.066664,0.030977,0.072794,0.288604,-0.158939,1,10419,critical
4,-0.077614,-0.044658,-0.158415,0.108990,0.041627,0.138106,-0.004088,0.092520,-0.016285,-0.078490,...,-0.018110,0.252501,-0.136523,0.109989,0.062893,0.293013,-0.095985,1,12232,critical


In [24]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/codeT5_embeddings/processed_embeddings/codeT5_base_embed_line_by_line_avg.pkl'
pd.to_pickle(data, dataset_path)